In [1]:
from selenium import webdriver  # Utilizado para automatizar interacciones del navegador web.
from selenium.webdriver.common.by import By  # Ayuda a localizar elementos dentro de una página web usando métodos como ID, XPATH, etc.
from selenium.webdriver.support.ui import WebDriverWait  # Permite la implementación de esperas explícitas que esperan a que ciertas condiciones ocurran antes de continuar.
from selenium.webdriver.support import expected_conditions as EC  # Se utilizan para definir condiciones específicas como la visibilidad de elementos.
from webdriver_manager.chrome import ChromeDriverManager  # Automatiza la gestión del driver de Chrome, asegurando que siempre esté actualizado.
from selenium.webdriver.support.ui import Select  # Facilita la interacción con los elementos <select> de HTML para seleccionar opciones.
import unicodedata  # Proporciona herramientas para trabajar con datos de texto Unicode, como normalizar formatos de texto.
import pandas as pd  # Utilizada para la manipulación y análisis de datos estructurados.
import requests  # Permite enviar solicitudes HTTP/1.1 de manera fácil, usada para interactuar con APIs o fetch de datos desde internet.
import csv  # Utilizada para leer y escribir en archivos CSV, facilita la manipulación de datos tabulares.
import time # Proporciona funciones para trabajar con tiempo, incluyendo funciones de pausa (sleep) y medición de tiempos.
import re

Se importan las librerías necesarias para manejar el navegador de manera automatizada con Selenium, realizar solicitudes HTTP con requests, manejar datos con pandas, y otras utilidades como csv y unicodedata para trabajar con archivos y texto, respectivamente.

### Configuración de Selenium, apertura de la página web y extracción de datos

In [3]:
# Configuración inicial de Selenium para usar Chrome con opciones predeterminadas
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
# URL del sitio web de Cinépolis para la cartelera de Cd. Juárez
url = 'https://cinepolis.com/cartelera/cd-juarez'
# Abre la URL en el navegador
driver.get(url)
# Diccionario que relaciona los nombres de los cines con sus valores en la página web
cines = {
    "Gran Patio Zaragoza": "cinepolis-gran-patio-zaragoza",
    "Juárez Centro Mall": "cinepolis-juarez-centro-mall",
    "La Monumental": "cinepolis-la-monumental",
    "Las Américas Juárez": "cinepolis-las-americas-juarez",
    "Las Misiones": "cinepolis-las-misiones",
    "Pinocelli": "cinepolis-pinocelli",
    "Plaza Areno": "cinepolis-plaza-areno",
    "Plaza Sendero Las Torres": "cinepolis-plaza-sendero-las-torres",
    "Sendero Ciudad Juárez": "cinepolis-sendero-ciudad-juarez",
    "VIP Las Misiones": "cinepolis-vip-las-misiones"
}

try:
    # Espera hasta que el botón para volver al sitio clásico se habilite para hacerle click y lo presiona
    volver_al_sitio_clasico = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Volver al sitio clásico')]"))
    )
    volver_al_sitio_clasico.click()
    # Espera hasta que el elemento del menú desplegable de ciudades esté presente para poder seleccionar Cd. Juarez
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "cmbCiudades"))
    )
    select_ciudad_element = driver.find_element(By.ID, "cmbCiudades")
    select_ciudad = Select(select_ciudad_element)
    select_ciudad.select_by_visible_text('Cd. Juárez')
    
    # Inicializa un diccionario para almacenar la información de las películas
    peliculas_info = {}

    # Iterar a través de cada cine en el diccionario
    for cine_nombre, cine_valor in cines.items():

        # Espera hasta que el menú de cines esté presente
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "cmbComplejos"))
        )
        select_cine_element = driver.find_element(By.ID, "cmbComplejos")
        select_cine = Select(select_cine_element)
        select_cine.select_by_value(cine_valor)

        time.sleep(5)  # Aseguramos la carga completa esperando 5 segundos

        # Recogemos información de títulos, clasificaciones, títulos originales y duraciones
        titulos = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.datalayer-movie.ng-binding'))
        )
        clasificaciones = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.clasificacion.ng-binding'))
        )
        titulos_originales = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.data-layer'))
        )
        duraciones = WebDriverWait(driver, 30).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.duracion.ng-binding'))
        )
        # Procesa cada película obtenida y almacena su información
        for titulo, clasificacion, titulo_original, duracion in zip(titulos, clasificaciones, titulos_originales, duraciones):
            # Extraemos y limpiamos el texto de los atributos, eliminando espacios en blanco al inicio y al final
            titulo_text = titulo.text.strip() 
            clasificacion_text = clasificacion.text.strip()
            titulo_original_text = titulo_original.get_attribute('data-titulooriginal').split(" (")[0] # Extrae el valor del atributo 'data-titulooriginal', lo divide en " (" y toma la primera parte para obtener solo el título principal
            duracion_text = duracion.text.replace(" min", "").strip()  # Limpia la duración y elimina " min"
            # Verifica si cualquiera de los textos extraídos está vacío, y si es así, salta al próximo ciclo del bucle
            if not titulo_text or not clasificacion_text or not titulo_original_text:
                continue

            if titulo_text not in peliculas_info:
                # Si el título no está, inicializa un nuevo diccionario para esa película con su información básica y un diccionario interno para contar presencias en cines
                peliculas_info[titulo_text] = {
                    'titulo_original': titulo_original_text,
                    'clasificacion': clasificacion_text,
                    'duracion': duracion_text,  # Guarda la duracion limpia solo en numeros
                    **{c: 0 for c in cines.keys()} # Crea un contador inicializado en 0 para cada cine
                }
            # Marca que la película está presente en el cine actual incrementando el contador en 1
            peliculas_info[titulo_text][cine_nombre] = 1
# Captura cualquier excepción que ocurra durante la ejecución del bloque "try"
except Exception as e:
    print(f"Error al procesar: {e}")
# Asegura que el navegador se cierre al finalizar la ejecución.
finally:
    driver.quit()


En este codigo Selenium automatiza la interacción con la página de cartelera de cine de Cinépolis en Cd. Juárez. Inicia configurando y lanzando un navegador Chrome, y navega a la URL específica. Utiliza esperas explícitas para manejar elementos dinámicos, como el botón "Volver al sitio clásico", que hace clic para asegurar la interfaz clásica, donde todos los datos necesarios estan disponibles. Posteriormente, refresca y selecciona opciones específicas de ciudad y cine de menús desplegables, asegurando recargar cada elemento antes de interactuar para evitar referencias obsoletas. Además, espera hasta que todos los elementos relevantes como títulos, clasificaciones y títulos originales estén presentes en la página, utilizando un timeout de 30 segundos para asegurarse de que la página haya cargado completamente. Se recopila la información de las películas, garantizando que no haya datos faltantes. El script finaliza cerrando el navegador para liberar recursos, asegurando un cierre adecuado del proceso de automatización.

### Crear DataFrame para visualización inicial de los datos y guardar los datos extraídos en un archivo CSV

In [4]:
# Se convierte el diccionario 'peliculas_info' en un DataFrame de pandas. Se creaa una lista de diccionarios, donde cada clave del diccionario original se convierte en una columna 'titulo', y el resto de los datos se mantiene como columnas adicionales.
df = pd.DataFrame([{'titulo': k, **v} for k, v in peliculas_info.items()])

# Se eliminan las filas duplicadas en el DataFrame para asegurar que cada película aparece solo una vez.
df = df.drop_duplicates()

# Se guarda el DataFrame en un archivo CSV llamado 'peliculas_cartelera.csv', sin guardar el índice del DataFrame y utilizando la codificación 'utf-8-sig' que es útil para mantener la integridad de los caracteres especiales en el CSV.
df.to_csv('peliculas_cartelera.csv', index=False, encoding='utf-8-sig')

# Se imprime un mensaje para confirmar que los datos han sido guardados exitosamente en el archivo especificado.
print("Datos guardados exitosamente en 'peliculas_cartelera.csv'.")

# Se muestra el DataFrame 'df' para visualizar su contenido y confirmar que los datos están correctos y listos.
print(df)


Datos guardados exitosamente en 'peliculas_cartelera.csv'.
                                               titulo  \
0                                  Amigos Imaginarios   
1                             El Hombre de Los Sueños   
2               El Planeta de Los Simios: Nuevo Reino   
3                                          Firma Aquí   
4                             Garfield: Fuera de Casa   
5                    Godzilla Y Kong:El Nuevo Imperio   
6           Gurren Lagann:Childhood's End,La película   
7               Monkey Man: El Despertar de La Bestia   
8                                   Profesión Peligro   
9                                  Tarot de La Muerte   
10                                        V de Victor   
11                             Jugaremos En El Bosque   
12                                              Frida   
13                                        Desafiantes   
14  Gurren Lagann: The Lights in the Sky Are Stars...   
15                           

Esta celda de código transforma un diccionario peliculas_info, que contiene información sobre películas, en un DataFrame de pandas. Cada entrada del diccionario se convierte en una fila del DataFrame, donde la clave del diccionario es el título de la película y sus valores asociados se convierten en columnas adicionales. Posteriormente, elimina cualquier fila duplicada para asegurar la unicidad de cada entrada. Luego, el DataFrame resultante se guarda en un archivo CSV titulado 'peliculas_cartelera.csv', sin incluir los índices y utilizando la codificación 'utf-8-sig' para mantener la correcta visualización de caracteres especiales. Al finalizar, imprime un mensaje confirmando que los datos han sido guardados correctamente y muestra el contenido del DataFrame para verificación. Este proceso es crucial para documentar y analizar la disponibilidad de películas en cines específicos.

### Funciones de apoyo para integración con API de TMDb

In [5]:
api_key = "b88b2a0c6be96e41cfd258225d190f0b"

Se establece una variable api_key con la clave de API necesaria para autenticarse y hacer solicitudes a la API de The Movie Database (TMDb). Esta clave es esencial para acceder a los recursos de la API que requieren autenticación y se utiliza en las siguientes funciones para realizar las peticiones HTTP pertinentes.

### Función para Normalizar Texto

In [6]:
def normalizar_texto(texto):
    # Convertir texto a minúsculas y remover tildes
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('ascii')
    texto = texto.lower()
    texto = texto.replace(" ", "_")
    return texto

Esta función toma una cadena de texto y la normaliza: convierte el texto a minúsculas, elimina caracteres especiales (como tildes) y reemplaza los espacios por guiones bajos. Es útil para preparar texto para consultas consistentes, especialmente cuando los nombres de género de películas necesitan ser estandarizados antes de ser utilizados en URLs o claves.

### Función para Obtener el Mapeo de Géneros de Películas

In [7]:
def obtener_mapeo_generos():
    # Construye la URL para obtener los géneros de películas desde TMDb usando la clave de API y especificando el idioma español
    url_generos = f"https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=es-ES"
    # Se realiza la solicitud HTTP GET a la API
    response = requests.get(url_generos)  
    # Se verifica si la respuesta es exitosa
    if response.status_code == 200:
        # Se extrae la lista de géneros del JSON de respuesta
        generos = response.json()['genres']  
        # Se inicializa un diccionario para almacenar el mapeo de géneros
        mapeo_generos = {}
        # Se itera sobre cada género
        for genero in generos:  
            # Se normaliza el nombre del género
            clave_normalizada = normalizar_texto(genero['name'])
            # Se excluye los géneros tipo 'Película de TV'
            if clave_normalizada != "pelicula_de_tv":  
                # Se asigna un prefijo y guarda en el diccionario
                mapeo_generos[genero['id']] = f"p_{clave_normalizada}" 
        return mapeo_generos  # Se devuelve el diccionario de mapeo de géneros
    else:
        raise ValueError("Error al obtener los géneros desde TMDb")  


Esta función se conecta con la API de The Movie Database para obtener un listado de géneros de películas, los normaliza para tener un formato consistente y los almacena en un diccionario. Este mapeo de géneros es útil para categorizar películas en aplicaciones y análisis de datos.

### Funcion para obtener detalles de las peliculas

In [8]:
def obtener_detalle_pelicula(titulo, idioma):
    url_busqueda = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={titulo}&language={idioma}"
    response = requests.get(url_busqueda)
    if response.status_code == 200:
        # Extrae la lista de resultados del JSON de respuesta
        resultados = response.json().get('results', [])
        if resultados:
            # Devuelve el primer resultado si existe
            return resultados[0]  
    # Devuelve None si no hay resultados o la respuesta no es exitosa
    return None

Esta función realiza una búsqueda en la API de The Movie Database utilizando el título de una película y un idioma específico. Retorna el primer resultado de la búsqueda, proporcionando detalles de la película que pueden incluir datos como sinopsis, clasificaciones, y más, lo cual es esencial para aplicaciones que requieren información detallada sobre películas específicas.

### Función para buscar y compilar detalles de películas

In [9]:
def buscar_detalle_pelicula(titulo_original, titulo_espanol, mapeo_generos):
    # Intenta obtener detalles de la película original en inglés
    resultado = obtener_detalle_pelicula(titulo_original, "en-US")
    if not resultado:
        # Si no encuentra en inglés, busca en español
        resultado = obtener_detalle_pelicula(titulo_espanol, "es-ES")

    if resultado:
        # Inicializa un diccionario para los géneros de la película con valores en 0
        generos_pelicula = {gen: 0 for gen in mapeo_generos.values()} 
        # Recorre los géneros de la película obtenida y actualiza el diccionario si el género está presente
        for gen_id in resultado['genre_ids']:
            if gen_id in mapeo_generos:
                generos_pelicula[mapeo_generos[gen_id]] = 1  # Marca el género como presente
        # Obtiene el promedio de votos y lo convierte a una escala de 100
        user_score = resultado.get('vote_average', 0) * 10 
        # Devuelve una lista de valores de géneros y el puntaje de usuario
        return list(generos_pelicula.values()) + [user_score]

    # Devuelve una lista de ceros para los géneros y el puntaje si no se encuentra información de la película
    return [0] * len(mapeo_generos) + [0]


Esta función busca los detalles de una película tanto por su título original en inglés como por su título en español utilizando la API de The Movie Database. Se enfoca en recopilar la información sobre los géneros de la película y su puntuación de usuario. Si se encuentra la película, devuelve una lista de géneros indicando la presencia (1) o ausencia (0) de cada género, seguido de la puntuación de usuario ajustada a una escala de 100. Si no se encuentran resultados, la función devuelve una lista de ceros para los géneros y la puntuación. Esta estructura de datos es útil para análisis o aplicaciones que necesiten clasificar películas según su contenido de género y popularidad.

### Preparación y Carga de Datos

In [10]:
# Leer el archivo CSV inicial
df_csv = pd.read_csv('peliculas_cartelera.csv')
datos_peliculas = []

# Obtener el mapeo de géneros
mapeo_generos = obtener_mapeo_generos()

Se cargan los datos iniciales de las películas desde un archivo CSV y de obtener el mapeo de géneros utilizando una función que accede a la API de The Movie Database. Esto prepara el entorno con los datos necesarios para el procesamiento y análisis siguiente

### Procesamiento de Datos

In [11]:
# Procesar cada fila del archivo CSV
for _, fila in df_csv.iterrows():
    titulo = fila['titulo']
    titulo_original = fila['titulo_original']
    clasificacion = fila['clasificacion']
    duracion = fila['duracion']
    detalles_pelicula = buscar_detalle_pelicula(titulo_original, titulo, mapeo_generos)
    detalles_pelicula += [fila[cine] for cine in df_csv.columns if cine not in ['titulo', 'titulo_original', 'clasificacion', 'duracion']]
    datos_peliculas.append([titulo, clasificacion, titulo_original, duracion] + detalles_pelicula)

Se procesa cada registro del DataFrame original para enriquecer los datos con información detallada sobre géneros y puntuaciones obtenidas a través de la API, y se agregan a una lista para posterior creación de un nuevo DataFrame

### Creación y almacenamiento del DataFrame del catalogo de peliculas para procesamiento

In [12]:
# Crear la lista de nombres de columnas, omitiendo el prefijo "Cinépolis " y normalizando el texto
cines_columnas = [(cine.replace("Cinépolis ", "")) for cine in df_csv.columns if cine not in ['titulo', 'titulo_original', 'clasificacion', 'duracion']]
columnas = ['titulo', 'clasificacion', 'titulo_original', 'duracion'] + list(mapeo_generos.values()) + ['score'] + cines_columnas

Se crea una lista de nombres de columnas para un DataFrame, omitiendo el prefijo "Cinépolis " y normalizando el texto (convirtiéndolo a minúsculas, eliminando tildes y reemplazando espacios con guiones bajos). La lista resultante incluye columnas para títulos, clasificaciones, títulos originales, duración, géneros de películas, puntuación y cines.

In [13]:
# Crear el DataFrame con todas las columnas necesarias
df_peliculas = pd.DataFrame(datos_peliculas, columns=columnas)

# Redondear el score a un número entero
df_peliculas['score'] = df_peliculas['score'].round().astype(int)

# Mostrar todas las columnas y filas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df_peliculas)

# Guardar el DataFrame en un archivo CSV
df_peliculas.to_csv('peliculas_clasificadas.csv', index=False, encoding='utf-8-sig')
print("La tabla ha sido guardada en 'peliculas_clasificadas.csv'.")

                                               titulo clasificacion  \
0                                  Amigos Imaginarios             A   
1                             El Hombre de Los Sueños           B15   
2               El Planeta de Los Simios: Nuevo Reino             B   
3                                          Firma Aquí             B   
4                             Garfield: Fuera de Casa            AA   
5                    Godzilla Y Kong:El Nuevo Imperio             B   
6           Gurren Lagann:Childhood's End,La película             B   
7               Monkey Man: El Despertar de La Bestia             C   
8                                   Profesión Peligro             B   
9                                  Tarot de La Muerte           B15   
10                                        V de Victor             A   
11                             Jugaremos En El Bosque           B15   
12                                              Frida             A   
13    

Consolidamos los datos procesados en un nuevo DataFrame, realizamos limpiezas finales como el redondeo del score, y finalmente guardamos el DataFrame en un nuevo archivo CSV para su uso o análisis futuros. También se configura la visualización en pandas para facilitar la revisión de los datos completos.